# ConvLSTM Model #7 with Heuristics
(results)

***LOAD THE FUNCTIONS AND PACKAGES***

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from tabulate import tabulate
from diniz_ftsv2 import *

Using TensorFlow backend.


***LOAD DATASET (Confidential)***

In [2]:
df = pd.read_csv('./dataset_.csv')
X = df.values
X = X.astype('float32')

***DATA COMPOSITION (raw data)***

X is formed by four input variables, which have been shifted at time *t=120* as described in the article. Consequently, each row of X consists of 480 samples (120x4) along with two columns that represent the output. The output, indicating either *clogging* or *non-clogging* is represented by the values **[1 0]** and **[0 1]**, respectively. Now, we will introduce the four input variables, the output, and the general parameters.

***1) Summary Statistics of Inputs:***


In [3]:
Xp = pd.DataFrame(X)
inputs = pd.DataFrame({'Casting Spd':Xp[0],'Mold Lvl':Xp[1],'Gate Opn.':Xp[2],'Tundish':Xp[3]})
outputs = pd.DataFrame({'Clogging':Xp[Xp.shape[1]-2], 'Normal':Xp[Xp.shape[1]-1]})

inputs.describe()


,Casting Spd,Mold Lvl,Gate Opn.,Tundish
count,3.054426e+06,3.054426e+06,3.054426e+06,3.054426e+06
mean,1.188411e+00,1.007837e+02,7.063924e+01,5.513210e+00
std,2.208997e-01,8.715239e+00,4.749716e+00,4.998255e-01
min,0.000000e+00,3.416350e+01,-2.888620e-02,5.000000e+00
25%,1.008350e+00,9.938640e+01,6.753560e+01,5.000000e+00
50%,1.180000e+00,1.000180e+02,7.087970e+01,6.000000e+00
75%,1.350000e+00,1.006580e+02,7.377410e+01,6.000000e+00
max,1.710000e+00,2.000000e+02,1.023190e+02,6.000000e+00


***2) First 5 Rows of Input (t=0):***

In [4]:
inputs.head()

,Casting Spd,Mold Lvl,Gate Opn.,Tundish
0,1.20000,100.542000,70.076103,5.0
1,1.19254,100.589996,68.787804,5.0
2,1.19725,100.617996,68.397697,5.0
3,1.19314,100.646004,69.981903,5.0
4,1.19000,101.015999,69.698196,5.0


***3) Presentation of the % of samples classified as Clogging and Normal:***

In [5]:
n_clog = outputs.Clogging[outputs.Clogging == 1].sum()  
n_normal = outputs.Normal[outputs.Normal == 1].sum()  

table = [['Output', '%'], 
         ['Clogging', round(100*n_clog/(n_clog+n_normal))], 
         ['Non-Clogging', round(100*n_normal/(n_clog+n_normal))]]
print(tabulate(table))

------------  --
Output        %
Clogging      7
Non-Clogging  93
------------  --


***4) First 5 Rows of Output:***

In [6]:
outputs.head()

,Clogging,Normal
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [7]:
del df, Xp, table, inputs, n_clog, n_normal, outputs

**PARAMETER DETERMINATION FOR MODEL TRAINING (as Reported in Article - ConvLSTM #7)**

* **delay** - determines the temporal sample size of each input matrix (as explained in the manuscript)
&nbsp;

* **n_features** - number of input variables (as explained in the manuscript)

***1) Defines the parameters used in the network, where:***
   
* **ftr_ly_01** - first layer filters
&nbsp;

* **ftr_ly_02** - second layer filters
&nbsp;

* **krn_ly_01** - first layer kernels
&nbsp;

* **krn_ly_02** - second layer kernels
&nbsp;

* **dropout_01** - first dropout layer
&nbsp;

* **dense_ly** - dense layer neurons
&nbsp;

* **dropout_02** - second dropout layer


***2) Defines the lists that will receive the results of the performance criteria of each repetition of the code:***

* **accuracy_**  - to save the accuracy values for each round
&nbsp;

* **precision_** - to save the precision values for each round
&nbsp;

* **Recall_** - to save the recall values for each round
&nbsp;

* **f1_** - to save the F1 values for each round
&nbsp;

* **MCC_** - to save the MCC values for each round

In [8]:
delay = 120
n_features = 4 

# 1)
ftr_ly_01 = 256 
ftr_ly_02 = 16 
krn_ly_01 = 5 
krn_ly_02 = 3 
dropout_01 = 0.5 
dense_ly = 200 
dropout_02 = 0.3 

# 2)
accuracy_ = list() 
precision_ = list() 
Recall_ = list() 
f1_ = list() 
MCC_ = list() 


***NEURAL NETWORK TRAINING***

***1) Split the dataset into training, validation, and testing sets***

* **X_train** - 80% (model training)
* **X_valid** - 20% (model validation)
* **X_test** - 20% (model tests)

***2) Defines the number of times the network will be retrained/tested (a kind of k-fold, with k=5)***
&nbsp;

**rounds** = 5

***3) Save the model and the test dataset (X_test and y_test)***
&nbsp;

where **model** is the Convlstm model trained


In [9]:
rounds = 5

for r in range(rounds):
    
    print('ConvLstm network training - START - round = %d' % (r+1))
    accuracy_test, precision_test, recall_test, f1_test, MCC_test, model, X_test, y_test = traintest_convlstm(X, r, delay, n_features, ftr_ly_01, ftr_ly_02, krn_ly_01, krn_ly_02, dropout_01, dense_ly, dropout_02)
    print('ConvLstm network training - CONCLUDED - round = %d' % (r+1))

    #presents the results of the test number "r" of the ConvLSTM(256,16) network:
    print('Performance parameters of the ConvLstm network (test) - round = %d' % (r+1))
    summarize_results_point(accuracy_test, precision_test, recall_test, f1_test, MCC_test)

    # groups the results of each network test
    accuracy_.append(accuracy_test)
    precision_.append(precision_test)
    Recall_.append(recall_test)
    f1_.append(f1_test)
    MCC_.append(MCC_test)  

    if r == 1:
        # model.save('convlstm_save.h5') # Salve model (optional)
        model_r = model
        Xtst = X_test
        ytst = y_test

    else:  
        del  model # delete model

ConvLstm network training - START - round = 1
40578/40578 [==============================] - 13s 331us/step
ConvLstm network training - CONCLUDED - round = 1
Performance parameters of the ConvLstm network (test) - round = 1
Accuracy 	  	 	 	 	 95.005%
Precision 	 	 	 	 	 95.213%
Recall 	 	 	 	 	 	 97.077%
F1 	 	 	 	 	 	 96.106%
MCC 	 	 	 	 	 	 0.901
ConvLstm network training - START - round = 2
40578/40578 [==============================] - 12s 293us/step
ConvLstm network training - CONCLUDED - round = 2
Performance parameters of the ConvLstm network (test) - round = 2
Accuracy 	  	 	 	 	 94.980%
Precision 	 	 	 	 	 95.088%
Recall 	 	 	 	 	 	 97.176%
F1 	 	 	 	 	 	 96.088%
MCC 	 	 	 	 	 	 0.900
ConvLstm network training - START - round = 3
40578/40578 [==============================] - 14s 353us/step
ConvLstm network training - CONCLUDED - round = 3
Performance parameters of the ConvLstm network (test) - round = 3
Accuracy 	  	 	 	 	 94.941%
Precision 	 	 	 	 	 95.904%
Recall 	 	 	 	 	

***TRAINING COMPLETED***

Presents the final results after testing the ConvLSTM(256,16) network:

In [10]:
summarize_results(accuracy_, precision_, Recall_, f1_, MCC_)

Accuracy 	  	 	 	 	 95.056% (+/-0.101)
Precision 	 	 	 	 	 95.464% (+/-0.288)
Recall 	 	 	 	 	 	 97.128% (+/-0.106)
F1 	 	 	 	 	 	 96.156% (+/-0.090)
MCC 	 	 	 	 	 	 0.902 (+/-0.002)


***POST-PROCESSING HEURISTICS***

Following the manuscript's explanation, the window size is set to 40 samples (*window = 40*), with minimum (*lim_min*) and maximum (*lim_max*) thresholds set to 0.3 and 0.8, respectively.

***1) Reload the model as well as the test set***

In [11]:
X_test = Xtst
y_test = ytst
model = model_r 

model.summary() # visualize model

Model: "ConvLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, 4, 1, 26, 256)     1332224   
_________________________________________________________________
conv_lst_m2d_4 (ConvLSTM2D)  (None, 1, 24, 16)         52288     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 24, 16)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               77000     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 402

***2) Apply the test set to the network using heuristics***

In [14]:
accuracy_h, precision_h, recall_h, f1_h, MCC_h = convlstm_heuristic(X_test, y_test, model)

***OVERALL CONVLSTM 7 RESULTS AFTER POST-PROCESSING HEURISTICS***

In [15]:
print('After Heuristic:')
summarize_results_point(accuracy_h, precision_h, recall_h, f1_h, MCC_h)

After Heuristic:
Accuracy 	  	 	 	 	 99.869%
Precision 	 	 	 	 	 97.938%
Recall 	 	 	 	 	 	 98.616%
F1 	 	 	 	 	 	 98.276%
MCC 	 	 	 	 	 	 0.982
